In [3]:
import pandas as pd
import numpy as np

path =('../../dataset/')

# 장르 결측치 처리 된 거 이후부터 시작!
movies_data = pd.read_csv('movies_genre_crawling.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [5]:
movie_dataset = movies_data.copy()

In [6]:
links_data = pd.read_csv(path+'links.csv')
links_data.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
movies = pd.merge(movie_dataset, links_data, on='movieId')

## 크롤링 준비
imdb_id를 url에 집어 넣는 형태로 변환

In [8]:
def create_imdb_url(imdb_id):
    # imdbId를 문자열로 변환하고, 7자리가 되도록 앞에 0을 추가
    imdb_id_str = str(imdb_id).zfill(7)
    # URL 생성
    url = f"https://www.imdb.com/title/tt{imdb_id_str}/"
    return url

In [ ]:
# 예시
imdb_id = 114709
url = create_imdb_url(imdb_id)
print(url)

## 1개로 연습

In [ ]:
from selenium import webdriver

browser = webdriver.Chrome()
browser.get('https://www.google.com/')

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(browser.page_source, 'html.parser')

In [64]:
names = movie_dataset['title'].tolist()
name = names[465]

In [65]:
imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
imdb_id

111127

In [66]:
imdb_url = create_imdb_url(imdb_id)

In [67]:
# IMDb 페이지로 이동
browser.get(imdb_url)

In [68]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [69]:
# 스크롤 내리기
from selenium.webdriver.common.keys import Keys

body = browser.find_elements('css selector', 'body')[0]
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)
body.send_keys(Keys.PAGE_DOWN)

soup = BeautifulSoup(browser.page_source, 'html.parser')

import time
time.sleep(2)

soup = BeautifulSoup(browser.page_source, 'html.parser')

In [70]:
soup = BeautifulSoup(browser.page_source, 'html.parser')
soup = BeautifulSoup(browser.page_source, 'html.parser')

storyline = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
storyline[0].text

"A picture perfect middle class family is shocked when they find out that one of their neighbors is receiving obscene phone calls. The mom takes slights against her family very personally, and it turns out she is indeed the one harassing the neighbor. As other slights befall her beloved family, the body count begins to increase, and the police get closer to the truth, threatening the family's picture perfect world. —Ed Sutton <esutton@mindspring.com>"

In [71]:
country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
country.text

'United States'

In [72]:
language_select = ''
language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')

# 모든 <ul> 태그를 순회
for ul in language_select:
    # 각 <ul> 태그 내의 모든 <li> 태그를 순회
    for li in ul.find_all('li'):
        span = li.find('span', class_='ipc-metadata-list-item__label')
        if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
            # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
            languages = [a.get_text(strip=True) for a in li.find_all('a')]
            languages_str = ', '.join(languages)
            print(languages_str)
            break
    else:
        # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
        continue
    # 'Language' 라벨을 찾은 경우, 반복문 종료
    break
else:
    print("No language information found")


English


In [73]:
a = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0]
a.text

'John Waters'

In [74]:
director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
director

'John Waters'

In [75]:
release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
release_year

'1994'

In [76]:
temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
for tag in temp:
    if 'hour' in tag.text or 'minutes' in tag.text:
        runtime = tag.text
        print(runtime)

1 hour 35 minutes


In [77]:
age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')[0].text
age_rating

'18'

In [78]:
tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(24) > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
for i in range(3):
    print(tags[i].text)

serial killer
dysfunctional family
serial killer as protagonist


# 전체 데이터셋을 대상으로 크롤링 후 저장!

In [9]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

names = movie_dataset['title'].tolist()

In [10]:
from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]
for name in tqdm(names[:500], desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
   
    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    time.sleep(2.5)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movie_dataset.loc[movie_dataset['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
   
    ################################################################################
            
    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movie_dataset.loc[movie_dataset['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################

    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movie_dataset.loc[movie_dataset['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movie_dataset.loc[movie_dataset['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        
     ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'age_rating'] = age_rating

    except:
        un_age_rating.append(name)
        
     ################################################################################
    
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_1'] = tags[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_2'] = tags[1].text
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)
     
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')


Crawling movies: 100%|██████████████████████████████████████████████████████████| 500/500 [1:10:09<00:00,  8.42s/movie]


In [98]:
movie_dataset.loc[400:440]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
400,459,"Getaway, The (1994)",Action|Adventure|Crime|Drama|Romance|Thriller,Doc McCoy is put in prison because his partner...,United States,English,Roger Donaldson,1994,1 hour 55 minutes,18,NaN,NaN,NaN
401,460,Getting Even with Dad (1994),Comedy,"Ray, an ex-con and widower, is planning a coin...",United States,English,Howard Deutch,1994,1 hour 49 minutes,All,NaN,NaN,NaN
402,461,Go Fish (1994),Drama|Romance,"Max is a trendy, pretty, young lesbian, who is...",United States,English,Rose Troche,1994,1 hour 23 minutes,Rated R for strong sexuality and sexual dialogue,NaN,NaN,NaN
403,464,Hard Target (1993),Action|Adventure|Crime|Thriller,Natasha Binder comes to New Orleans looking fo...,United States,"English, French",John Woo,1993,1 hour 37 minutes,18,NaN,NaN,NaN
404,466,Hot Shots! Part Deux (1993),Action|Comedy|War,Topper Harley is found working as an odd-job-m...,United States,English,Jim Abrahams,1993,1 hour 26 minutes,15,NaN,NaN,NaN
405,467,Live Nude Girls (1995),Comedy,Georgina throws a sleepover party for her frie...,United States,English,Julianna Lavin,1995,1 hour 35 minutes,Rated R for graphic sexual dialogue and scenes...,NaN,NaN,NaN
406,468,Englishman Who Went Up a Hill But Came Down a ...,Comedy|Romance,Two English cartographers visit the small Sout...,United Kingdom,"English, Welsh",Christopher Monger,1995,1 hour 39 minutes,Rated PG for mild language,NaN,NaN,NaN
407,469,"House of the Spirits, The (1993)",Drama|Romance,"In mid-20th-century, the impoverished Esteban ...",Germany,"English, Spanish, French",Bille August,1993,2 hours 25 minutes,18,NaN,NaN,NaN
408,470,House Party 3 (1994),Comedy,"Come to a new House Party, where Kid, after a ...",United States,English,Eric Meza,1994,1 hour 40 minutes,Rated R for strong language and some strong se...,NaN,NaN,NaN
409,471,"Hudsucker Proxy, The (1994)",Comedy,"When Waring Hudsucker, head of hugely successf...",United Kingdom,English,Joel Coen,1994,1 hour 51 minutes,15,NaN,NaN,NaN


In [163]:
# null 값 확인
#print("storyline : ", un_storyline)
print("----------"*10)
print("country : ", un_country)
print("----------"*10)
print("language : ", un_language)
print("----------"*10)
print("director : ", un_director)
print("----------"*10)
#print("release_year : ", un_release_year)
print("----------"*10)
print("runtime : ", un_runtime)
print("----------"*10)
#print("age_rating : ", un_age_rating)
print("----------"*10)
print("tags : ", un_tags)
print("----------"*10)

----------------------------------------------------------------------------------------------------
country :  []
----------------------------------------------------------------------------------------------------
language :  []
----------------------------------------------------------------------------------------------------
director :  []
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
runtime :  []
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
tags :  ['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Farinelli: il castrato (1994)', 'Gordy (1995)', 'Hoop Dreams (1994)', 'Heavenly Creatures (1994)', 'Houseguest (1994)', 'Immortal Beloved (1994)', 'Heavyweights (Heavy W

In [164]:
len(un_tags)

290

In [165]:
# 저장 안된 값에 한해서 다시 진행 storyline
un_tags_2 = []
for name in tqdm(un_tags, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
   
    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    time.sleep(1.5)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'age_rating'] = age_rating

    except:
        un_tags_2.append(name)
        
    soup = BeautifulSoup(browser.page_source, 'html.parser')


Crawling movies:  78%|██████████████████████████████████████████████▉             | 227/290 [35:57<09:58,  9.51s/movie]


IndexError: list index out of range

In [160]:
movie_dataset.loc[movie_dataset['title'] == un_release_year[4], 'release_year'] = '1995' 

In [162]:
movie_dataset.loc[movie_dataset['title'] == un_release_year[5]]

IndexError: list index out of range

In [96]:
len(un_age_rating_4)

8

In [ ]:
movie_dataset_sample = movie_dataset[:5]

In [ ]:
movie_dataset.to_csv('movies_genre_storyline_ing.csv', index=False)


In [ ]:
# 출력할 때 최대 너비 설정
pd.set_option('display.max_colwidth', None)

movie_dataset.loc[1120:1130]['storyline']